In [4]:
import spacy
import pandas as pd

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Simpler gender map
GENDER_MAP = {
    "he": "she", "she": "he",
    "him": "her", "her": "him",
    "his": "her", "hers": "his",
    "himself": "herself", "herself": "himself"
}

def swap_gender_pronouns(text, target_gender="female"):
    doc = nlp(text)
    result_tokens = []

    for token in doc:
        tok_text = token.text
        lower = tok_text.lower()

        replacement = None

        if lower == "her":
            if token.dep_ == "poss":
                replacement = "his"
            else:
                replacement = "him"
        elif lower == "his":
            if token.dep_ == "poss":
                replacement = "her"
            else:
                replacement = "hers"
        else:
            replacement = GENDER_MAP.get(lower, None)

        if replacement:
            if tok_text.istitle():
                replacement = replacement.capitalize()
            result_tokens.append(replacement + token.whitespace_)
        else:
            result_tokens.append(tok_text + token.whitespace_)

    return "".join(result_tokens)



In [5]:
import pandas as pd
df = pd.read_csv("pronoun_testcases.csv")

# Add flipped sentences
target_gender = "female"  # or "male"
df["Flipped_Sentence"] = df["input_text"].apply(lambda x: swap_gender_pronouns(x))

# Display result
df.head(20)

,input_text,target_gender,expected_output,Flipped_Sentence
0,He is going to the market.,female,She is going to the market.,She is going to the market.
1,His book is on the table.,female,Her book is on the table.,Her book is on the table.
2,I saw him yesterday.,female,I saw her yesterday.,I saw her yesterday.
3,He hurt himself.,female,She hurt herself.,She hurt herself.
4,I called him last night.,female,I called her last night.,I called her last night.
5,That is his car.,female,That is her car.,That is her car.
6,He told me about his trip.,female,She told me about her trip.,She told me about her trip.
7,The teacher gave him a warning.,female,The teacher gave her a warning.,The teacher gave her a warning.
8,He blames himself for the mistake.,female,She blames herself for the mistake.,She blames herself for the mistake.
9,He brought his laptop.,female,She brought her laptop.,She brought her laptop.


In [6]:
# After your flipping logic
df["Match"] = df["Flipped_Sentence"].str.strip() == df["expected_output"].str.strip()

# Print mismatched rows
mismatches = df[~df["Match"]]
print("Accuracy: {:.2f}%".format(df["Match"].mean() * 100))
print("\nMismatched cases:")
print(mismatches[["input_text", "expected_output", "Flipped_Sentence"]])


Accuracy: 100.00%

Mismatched cases:
Empty DataFrame
Columns: [input_text, expected_output, Flipped_Sentence]
Index: []
